In [1]:
from utils.data_scrap_functions import *

Ce notebook est temporaire, il sera remplacé par un moyen plus simple et ergonomique d'importer les données.

## Importation des données

### Dans un premier temps, on importe la liste de toute les variables à récupérer

In [3]:
url = 'https://classic.clinicaltrials.gov/api/info/study_fields_list'

In [4]:
list_fields = get_list(url)

In [6]:
list_fields[:20]

['Acronym',
 'AgreementOtherDetails',
 'AgreementPISponsorEmployee',
 'AgreementRestrictionType',
 'AgreementRestrictiveAgreement',
 'ArmGroupDescription',
 'ArmGroupInterventionName',
 'ArmGroupLabel',
 'ArmGroupType',
 'AvailIPDComment',
 'AvailIPDId',
 'AvailIPDType',
 'AvailIPDURL',
 'BaselineCategoryTitle',
 'BaselineClassDenomCountGroupId',
 'BaselineClassDenomCountValue',
 'BaselineClassDenomUnits',
 'BaselineClassTitle',
 'BaselineDenomCountGroupId',
 'BaselineDenomCountValue']

### Maintenant, on importe les données depuis l'API de Clinical Trials selon la liste de variable qu'on veut

In [7]:
get_data(taille=10,list_fields=list_fields[-40:])

100%|██████████| 10/10 [00:35<00:00,  3.50s/it]
/home/onyxia/work/ProjetPython/utils/data_scrap_functions.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df=full_df.applymap(treat_list)


Temps requete API: -------------------- 34.56310057640076 

 Temps Pandas: -------------------- 0.4118835926055908


,RetractionSource,SamplingMethod,SecondaryId,SecondaryIdDomain,SecondaryIdLink,SecondaryIdType,SecondaryOutcomeDescription,SecondaryOutcomeMeasure,SecondaryOutcomeTimeFrame,SeeAlsoLinkLabel,SeeAlsoLinkURL,SeriousEventAssessmentType,SeriousEventNotes,SeriousEventOrganSystem,SeriousEventSourceVocabulary,SeriousEventStatsGroupId,SeriousEventStatsNumAffected,SeriousEventStatsNumAtRisk,SeriousEventStatsNumEvents,SeriousEventTerm,StartDate,StartDateType,StatusVerifiedDate,StdAge,StudyFirstPostDate,StudyFirstPostDateType,StudyFirstSubmitDate,StudyFirstSubmitQCDate,StudyPopulation,StudyType,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,None,None,None,None,None,None,Part A: Change from Week 12 in AOBP SBP at Wee...,Week 12 to Week 20,None,None,None,None,None,None,None,None,None,None,None,December 2023,Anticipated,November 2023,Adult,"November 29, 2023",Estimate,"November 17, 2023","November 27, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
1,None,Probability Sample,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"August 1, 2023",Actual,July 2023,Adult,"November 29, 2023",Estimate,"November 1, 2023","November 27, 2023",study population are orthodontic patients whic...,Observational,None,None,None,None,None,None,None,None,"November 29, 2023",None
2,None,None,None,None,None,None,Generalized Anxiety Disorder - 7 (GAD - 7) Anx...,Change in anxiety level at surgery from baseline,At surgery,None,None,None,None,None,None,None,None,None,None,None,"February 20, 2024",Anticipated,November 2023,Adult,"November 29, 2023",Estimate,"March 30, 2022","November 27, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
3,None,None,None,None,None,None,The incidence of AEs and SAE from screening to...,Safety - AEs and SAEs incidences over the stud...,from visit 1 to 24-months of safety and effica...,None,None,None,None,None,None,None,None,None,None,None,"April 30, 2024",Anticipated,November 2023,Adult,"November 29, 2023",Estimate,"November 13, 2023","November 27, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
4,None,None,None,None,None,None,Major adverse cardiac and cerebral events (MAC...,Major cardiac adverse events (MACCE),6-months,None,None,None,None,None,None,None,None,None,None,None,"December 4, 2023",Anticipated,November 2023,Adult,"November 29, 2023",Estimate,"November 20, 2023","November 27, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,August 2023,Anticipated,August 2023,Adult,"September 1, 2023",Actual,"February 17, 2022","August 26, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
9996,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"October 25, 2022",Actual,August 2023,Adult,"September 1, 2023",Actual,"August 26, 2023","August 26, 2023",None,Interventional,None,None,None,None,None,None,None,None,"November 29, 2023",None
9997,None,Probability Sample,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"September 5, 2019",Actual,September 2023,Adult,"September 1, 2023",Actual,"August 26, 2023","August 26, 2023",Healthy subjects and urology patients without ...,Observational,None,None,None,None,None,None,None,None,"November 29, 2023",None
9998,None,None,None,None,None,None,MRI protocol; Proton density (PD-TSE) sagittal...,MRI cartilage thickness,at baseline and 12th week,None,None,None,None,None,No

In [7]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import trange

In [8]:
def fetch_data(url):
    try:
        req_temp = requests.get(url)
        req_temp.raise_for_status()  # Vérifie si la requête a réussi
        data_temp = req_temp.json()['StudyFieldsResponse']['StudyFields']
        return pd.DataFrame(data_temp).drop('Rank', axis=1)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        time.sleep(3)  # Attend 3 secondes en cas d'erreur
        return pd.DataFrame()

def get_data_parralel(taille=10, list_fields=['NCTId', 'StartDate', 'LastUpdatePostDate'], keyword=''):
    full_df = pd.DataFrame()
    http_time = 0
    pandas_time = 0
    itter = len(list_fields) // 20 + 1

    with ThreadPoolExecutor(max_workers=itter) as executor:
        for j in trange(taille):
            futures = []

            for i in range(itter):
                fields = '%2C'.join(list_fields[i * 20:(i + 1) * 20])
                url_temp = f'https://classic.clinicaltrials.gov/api/query/study_fields?expr={keyword}&fields='
                url_temp += fields
                url_temp += f'&min_rnk={j * 1000 + 1}&max_rnk={(j + 1) * 1000}&fmt=json'

                futures.append(executor.submit(fetch_data, url_temp))

            st = time.time()
            dfs = [future.result() for future in futures]
            et = time.time()
            http_time += et - st

            st = time.time()
            df = pd.concat(dfs, axis=1)
            et = time.time()
            pandas_time += et - st

            st = time.time()
            full_df = pd.concat([full_df, df], axis=0)
            et = time.time()
            pandas_time += et - st

    full_df = full_df.reset_index(drop=True)
    full_df = full_df.applymap(treat_list)

    print("Temps requete API:", '-' * 20, http_time, '\n\n', 'Temps Pandas:', '-' * 20, pandas_time)
    return full_df

In [11]:
df = get_data_parralel(taille=100,list_fields=list_fields)
df

 67%|██████▋   | 67/100 [05:01<03:40,  6.67s/it]

Error fetching data from https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=OutcomeAnalysisParamValue%2COutcomeAnalysisStatisticalComment%2COutcomeAnalysisStatisticalMethod%2COutcomeAnalysisTestedNonInferiority%2COutcomeCategoryTitle%2COutcomeClassDenomCountGroupId%2COutcomeClassDenomCountValue%2COutcomeClassDenomUnits%2COutcomeClassTitle%2COutcomeDenomCountGroupId%2COutcomeDenomCountValue%2COutcomeDenomUnits%2COutcomeGroupDescription%2COutcomeGroupId%2COutcomeGroupTitle%2COutcomeMeasureAnticipatedPostingDate%2COutcomeMeasureCalculatePct%2COutcomeMeasureDenomUnitsSelected%2COutcomeMeasureDescription%2COutcomeMeasureDispersionType&min_rnk=67001&max_rnk=68000&fmt=json: 500 Server Error: Internal Server Error for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=OutcomeAnalysisParamValue%2COutcomeAnalysisStatisticalComment%2COutcomeAnalysisStatisticalMethod%2COutcomeAnalysisTestedNonInferiority%2COutcomeCategoryTitle%2COutcomeClassDenomCountGr

100%|██████████| 100/100 [07:53<00:00,  4.73s/it]
/tmp/ipykernel_3574/1518409255.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(treat_list)


Temps requete API: -------------------- 344.1433584690094 

 Temps Pandas: -------------------- 128.48616003990173


,Acronym,AgreementOtherDetails,AgreementPISponsorEmployee,AgreementRestrictionType,AgreementRestrictiveAgreement,ArmGroupDescription,ArmGroupInterventionName,ArmGroupLabel,ArmGroupType,AvailIPDComment,...,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,None,None,None,None,this group will be A control group receive no ...,Behavioral: no intervention,Group 4 (placebo),Placebo Comparator,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
1,None,None,None,None,None,Regimen: Norepinephrine increases of 0.05-0.1 ...,Other: Successive administration of vasopressors,STEPWISE VASOPRESSOR SEPTIC SHOCK MANAGEMENT,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
2,None,None,None,None,None,Zygomatic implant placement by dynamic compute...,Device: Freehand zygomatic implant placement,Navigation group,Experimental,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
3,None,None,None,None,None,"In tDCS, direct electrical currents at the int...",Device: transcranial direct current stimulatio...,tDCS group,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
4,None,None,None,None,None,Highly viscous glass ionomer restoration is us...,Procedure: Highly viscous glass ionomer (cervi...,Highly viscous Glass Ionomer,Experimental,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,None,None,None,None,None,Test drug (Ekmasonid) 1 tablet contains 9 mg B...,Drug: Ekmasonid,T test,Experimental,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
99996,None,None,None,None,None,None,Radiation: Pre reconstruction Radiotherapy,One Stage Reconstruction With pre-reconstructi...,None,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None
99997,DREAM-T2D,None,None,None,None,Group A will receive 1800 mg Metformin DR qAM ...,Drug: Metformin DR,Investigational Arm (Metformin DR plus metform...,Experimental,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",Strategic company decision - not related to a ...
99998,ELEVATE HNSCC,None,None,None,None,Participants with untreated metastatic or unre...,Drug: Magrolimab,"Safety Run-in Cohort 1, Magrolimab + Pembroliz...",Experimental,None,...,None,None,None,None,None,None,None,None,"December 04, 2023",None


In [16]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df.to_csv('./data/clini_data.csv')